<a href="https://colab.research.google.com/github/manhbd-22022602/big_data_for_beginners/blob/main/Hadoop_Setting_up_a_Single_Node_Cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://github.com/groda/big_data"><div><img src="https://github.com/groda/big_data/blob/master/logo_bdb.png?raw=true" align=right width="90"></div></a>

# HDFS and MapReduce on a single-node Hadoop cluster
<br>
<br>

We're going to set up a single-node cluster (following the instructions on https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/SingleCluster.html) and show how to run simple HDFS and MapReduce commands.

After downloading the software, it is necessary to carry out some preliminary steps like setting environment variables, generating SSH keys, etc.). We grouped all these steps under "Prologue".

Once done with the prologue, we are able to start a single-node Hadoop cluster on the current virtual machine.

We are going to run some test HDFS commands and MapReduce jobs on the Hadoop cluster.

Finally, the cluster will be shut down.


**TABLE OF CONTENTS**
* **[Prologue](#scrollTo=oUuQjW2oNMcJ)**

 * [Check the available Java version](#scrollTo=qFfOrktMPq8M)

 * [Download core Hadoop](#scrollTo=KE7kSYSXQYLf)

   * [Verify the downloaded file](#scrollTo=lGI4TNXPamMr)

 * [Configure `PATH`](#scrollTo=RlgP1ytnRtUK)

 * [Configure `core-site.xml` and `hdfs-site.xml`](#scrollTo=KLmxLQeJSb4A)

 * [Set environment variables](#scrollTo=kXbSKFyeMqr2)

 * [Setup localhost access via SSH key](#scrollTo=k2-Fdp73cF0V)

   * [Install `openssh` and start server](#scrollTo=-Uxmv3RdUwiF)

   * [Generate key](#scrollTo=PYKoSlaENuyG)

   * [Check SSH connection to localhost](#scrollTo=FwA6rKpScnVi)

* **[Launch a single-node Hadoop cluster](#scrollTo=V68C4cDySyek)**

   * [Initialize the namenode](#scrollTo=HTDPwnVlSbHS)

   * [Start cluster](#scrollTo=xMrEiLB_VAeR)

* **[Run some simple HDFS commands](#scrollTo=CKRRbwDFv3ZQ)**

* **[Run some simple MapReduce jobs](#scrollTo=G3KBe4R65bl1)**

   * [Simplest MapReduce job](#scrollTo=yVJA-3jSATGV)

   * [Another MapReduce example: filter a log file](#scrollTo=BbosNo0TD3oH)

   * [Aggregate data with MapReduce](#scrollTo=Sam22f-YT1xR)

* **[Stop cluster](#scrollTo=IF6-Z5RotAcO)**

* **[Concluding remarks](#scrollTo=w5N7tb0HSbZB)**



# Prologue

## Check the available Java version
 Apache Hadoop 3.3 and upper supports Java 8 and Java 11 (runtime only). See: https://cwiki.apache.org/confluence/display/HADOOP/Hadoop+Java+Versions


Check if Java version is one of `8`, `11`

In [43]:
!java -version

openjdk version "11.0.22" 2024-01-16
OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1)
OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1, mixed mode, sharing)


In [44]:
%%bash
JAVA_MAJOR_VERSION=$(java -version 2>&1 | grep -m1 -Po '(\d+\.)+\d+' | cut -d '.' -f1)
if [[ $JAVA_MAJOR_VERSION -eq 8 || $JAVA_MAJOR_VERSION -eq 11 ]]
 then
 echo "Java version is one of 8, 11 ✓"
 fi

Java version is one of 8, 11 ✓


Find the variable for the environment variable `JAVA_HOME`

Find the path for the environment variable `JAVA_HOME`

In [45]:
!readlink -f $(which java)

/usr/lib/jvm/java-11-openjdk-amd64/bin/java


Extract JAVA_HOME from the Java path by removing the `bin/java` part in the end

In [46]:
%%bash
JAVA_HOME=$(readlink -f $(which java) | sed 's/\/bin\/java$//')
echo $JAVA_HOME

/usr/lib/jvm/java-11-openjdk-amd64


We're going to use `JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64`.

Use `%env%` to set the variable for the current notebook session.

In [47]:
%env JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64

env: JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64


## Download core Hadoop
Download the latest stable version of the core Hadoop distribution from one of the download mirrors locations https://www.apache.org/dyn/closer.cgi/hadoop/common/.

**Note** with the option `--no-clobber`, `wget` will not download the file if it already exists.

In [48]:
!wget --no-clobber https://dlcdn.apache.org/hadoop/common/stable/hadoop-3.4.0.tar.gz

File ‘hadoop-3.4.0.tar.gz’ already there; not retrieving.



Uncompress archive

In [49]:
!tar xzf hadoop-3.4.0.tar.gz

### Verify the downloaded file

(see https://www.apache.org/dyn/closer.cgi/hadoop/common/)

Download sha512 file

In [50]:
! wget --no-clobber https://dlcdn.apache.org/hadoop/common/stable/hadoop-3.4.0.tar.gz.sha512

File ‘hadoop-3.4.0.tar.gz.sha512’ already there; not retrieving.



Compare

In [51]:
%%bash
A=$(sha512sum hadoop-3.4.0.tar.gz | cut - -d' ' -f1)
B=$(cut hadoop-3.4.0.tar.gz.sha512 -d' ' -f4)
printf "%s\n%s\n" $A $B
[[ $A == $B ]] && echo "True"

6f653c0109f97430047bd3677c50da7c8a2809d153b231794cf980b3208a6b4beff8ff1a03a01094299d459a3a37a3fe16731629987165d71f328657dbf2f24c
6f653c0109f97430047bd3677c50da7c8a2809d153b231794cf980b3208a6b4beff8ff1a03a01094299d459a3a37a3fe16731629987165d71f328657dbf2f24c
True


## Configure `PATH`

Add the Hadoop folder to the `PATH` environment variable


In [52]:
!echo $PATH

/content/hadoop-3.4.0/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin


In [53]:
import os
os.environ['HADOOP_HOME'] = os.path.join('/content', 'hadoop-3.4.0')
os.environ['PATH'] = ':'.join([os.path.join(os.environ['HADOOP_HOME'], 'bin'), os.environ['PATH']])
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'

In [54]:
import os
print(os.environ)

environ({'SHELL': '/bin/bash', 'NV_LIBCUBLAS_VERSION': '12.2.5.6-1', 'NVIDIA_VISIBLE_DEVICES': 'all', 'COLAB_JUPYTER_TRANSPORT': 'ipc', 'NV_NVML_DEV_VERSION': '12.2.140-1', 'NV_CUDNN_PACKAGE_NAME': 'libcudnn8', 'CGROUP_MEMORY_EVENTS': '/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events', 'NV_LIBNCCL_DEV_PACKAGE': 'libnccl-dev=2.19.3-1+cuda12.2', 'NV_LIBNCCL_DEV_PACKAGE_VERSION': '2.19.3-1', 'VM_GCE_METADATA_HOST': '169.254.169.253', 'HOSTNAME': '799b6dc81634', 'LANGUAGE': 'en_US', 'TBE_RUNTIME_ADDR': '172.28.0.1:8011', 'COLAB_TPU_1VM': '', 'GCE_METADATA_TIMEOUT': '3', 'NVIDIA_REQUIRE_CUDA': 'cuda>=12.2 brand=tesla,driver>=470,driver<471 brand=unknown,driver>=470,driver<471 brand=nvidia,driver>=470,driver<471 brand=nvidiartx,driver>=470,driver<471 brand=geforce,driver>=470,driver<471 brand=geforcertx,driver>=470,driver<471 brand=quadro,driver>=470,driver<471 brand=quadrortx,driver>=470,driver<471 brand=titan,driver>=470,driver<471 brand=titanrtx,driver>=470,dr

In [55]:
!echo $PATH

/content/hadoop-3.4.0/bin:/content/hadoop-3.4.0/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin


## Configure `core-site.xml` and `hdfs-site.xml`

Edit the file `etc/hadoop/core-site.xml` and `etc/hadoop/hdfs-site.xml` to configure pseudo-distributed operation.

**`etc/hadoop/core-site.xml`**
```
<configuration>
    <property>
        <name>fs.defaultFS</name>
        <value>hdfs://localhost:9000</value>
    </property>
</configuration>
```

**`etc/hadoop/hdfs-site.xml`**
```
<configuration>
    <property>
        <name>dfs.replication</name>
        <value>1</value>
    </property>
</configuration>
```

In [56]:
%%bash
echo -e "<configuration> \n\
    <property> \n\
        <name>fs.defaultFS</name> \n\
        <value>hdfs://localhost:9000</value> \n\
    </property> \n\
</configuration>" >hadoop-3.4.0/etc/hadoop/core-site.xml

echo -e "<configuration> \n\
    <property> \n\
        <name>dfs.replication</name> \n\
        <value>1</value> \n\
    </property> \n\
</configuration>" >hadoop-3.4.0/etc/hadoop/hdfs-site.xml

Check

In [57]:
cat hadoop-3.4.0/etc/hadoop/hdfs-site.xml

<configuration> 
    <property> 
        <name>dfs.replication</name> 
        <value>1</value> 
    </property> 
</configuration>


## Set environment variables

Add the following lines to the Hadoop configuration script `hadoop-env.sh`(the script is in `hadoop-3.4.0/sbin`).
```
export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64
export HDFS_NAMENODE_USER=root
export HDFS_DATANODE_USER=root
export HDFS_SECONDARYNAMENODE_USER=root
export YARN_RESOURCEMANAGER_USER=root
export YARN_NODEMANAGER_USER=root
```

In [58]:
%%bash
echo -e "export HDFS_NAMENODE_USER=root \n\
export HDFS_DATANODE_USER=root \n\
export HDFS_SECONDARYNAMENODE_USER=root \n\
export YARN_RESOURCEMANAGER_USER=root \n\
export YARN_NODEMANAGER_USER=root" >> hadoop-3.4.0/etc/hadoop/hadoop-env.sh

## Setup localhost access via SSH key

We are going to allow passphraseless access to `localhost` with a secure key.

SSH must be installed and sshd must be running to use the Hadoop scripts that manage remote Hadoop daemons.



### Install `openssh` and start server

I'm not sure why we need the option `StrictHostKeyChecking no`. This option tells the `ssh` server to allow key authentication only from known hosts, in particular it prevents a host from authenticating with key if the key has changed. I guess this option is needed since a new ssh key is generated every time one runs this notebook.

Alternatively, one could just delete the file `~/.ssh/known_hosts` or else use `ssh-keygen -R hostname` to remove all keys belonging to hostname from the `known_hosts` file (see for instance [How to remove strict RSA key checking in SSH and what's the problem here?](https://serverfault.com/questions/6233/how-to-remove-strict-rsa-key-checking-in-ssh-and-whats-the-problem-here) or [Remove key from known_hosts](https://superuser.com/questions/30087/remove-key-from-known-hosts)). The option `ssh-keygen -R hostname` seems the most elegant and I'm going to try it out some day.


In [59]:
%%bash
apt-get update
apt-get install openssh-server
echo 'StrictHostKeyChecking no' >> /etc/ssh/ssh_config
/etc/init.d/ssh restart

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
openssh-server is already the newest version (1:8.9p1-3ubuntu0.7).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
 * Restarting OpenBSD Secur

### Generate key
Generate SSH key that does not require a password.

The private key is contained in the file `id_rsa` located in the folder `~/.ssh`.

The public key is added to the file `~/.ssh/authorized_keys` in order to allow authentication with that key.

In [60]:
%%bash
rm /root/.ssh/id_rsa
ssh-keygen -t rsa -P '' -f ~/.ssh/id_rsa
cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys
chmod 0600 ~/.ssh/authorized_keys

Generating public/private rsa key pair.
Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in /root/.ssh/id_rsa.pub
The key fingerprint is:
SHA256:qfMgaSavapmbmMyLwFH2wsskvS6g/FM/+8/+dM7TQWQ root@799b6dc81634
The key's randomart image is:
+---[RSA 3072]----+
|                 |
|               E |
|   o          o  |
|  = .    .     . |
| o = .  S     .  |
|o = +o .       . |
|=.++* =      . .o|
|BBo* . *  . . +..|
|*B*+o  .=.o+.. o.|
+----[SHA256]-----+


### Check SSH connection to localhost

The following command should output "hi!" if the connection works.

In [61]:
!ssh localhost "echo hi!"

hi!


# Launch a single-node Hadoop cluster

## Initialize the namenode

In [62]:
!hdfs namenode -format -nonInteractive

namenode is running as process 1844.  Stop it first and ensure /tmp/hadoop-root-namenode.pid file is empty before retry.


In [63]:
!echo -e "export JAVA_HOME=$(readlink -f $(which java) | sed 's/\/bin\/java$//') \n\
export HDFS_NAMENODE_USER=root \n\
export HDFS_DATANODE_USER=root \n\
export HDFS_SECONDARYNAMENODE_USER=root \n\
export YARN_RESOURCEMANAGER_USER=root \n\
export YARN_NODEMANAGER_USER=root" >> hadoop-3.4.0/etc/hadoop/hadoop-env.sh

## Start cluster

In [64]:
!$HADOOP_HOME/sbin/start-dfs.sh

Starting namenodes on [localhost]
localhost: namenode is running as process 1844.  Stop it first and ensure /tmp/hadoop-root-namenode.pid file is empty before retry.
Starting datanodes
localhost: datanode is running as process 1949.  Stop it first and ensure /tmp/hadoop-root-datanode.pid file is empty before retry.
Starting secondary namenodes [799b6dc81634]
799b6dc81634: secondarynamenode is running as process 2137.  Stop it first and ensure /tmp/hadoop-root-secondarynamenode.pid file is empty before retry.


# Run some simple HDFS commands

In [23]:
%%bash
# create directory "my_dir" in HDFS home
hdfs dfs -mkdir /user
hdfs dfs -mkdir /user/root # this is the "home" of user root on HDFS
hdfs dfs -mkdir my_dir

# upload file mnist_test.csv to my_dir
hdfs dfs -put /content/sample_data/mnist_test.csv my_dir/

# show contents of directory my_dir
hdfs dfs -ls -h my_dir


Found 1 items
-rw-r--r--   1 root supergroup     17.4 M 2024-04-23 14:02 my_dir/mnist_test.csv


# Run some simple MapReduce jobs
We're going to use the [streaming](https://hadoop.apache.org/docs/stable/hadoop-streaming/HadoopStreaming.html) library.
WIth this utility any executable can be used as the mapper and/or the reducer.

## Simplest MapReduce job

This is a "no-code" example since we are going to use the existing Unix commands `cat` and `wc` respectively as mapper and as reducer. The result will show a line with three values: the counts of lines, words, and characters in the input file(s).

Input folder is `/user/my_user/my_dir/`, output folder `/user/my_user/output_simplest`.

**Note**: the output folder should not exist because it is created by Hadoop (this is in acordance with Hadoop's principle of not overwriting data).

Now run the MapReduce job

In [24]:
%%bash

hdfs dfs -rm -r output_simplest || hdfs namenode -format -nonInteractive
mapred streaming \
  -input my_dir \
  -output output_simplest \
  -mapper /bin/cat \
  -reducer /usr/bin/wc

rm: `output_simplest': No such file or directory
namenode is running as process 1844.  Stop it first and ensure /tmp/hadoop-root-namenode.pid file is empty before retry.
2024-04-23 14:02:51,348 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2024-04-23 14:02:51,518 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2024-04-23 14:02:51,518 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2024-04-23 14:02:51,543 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2024-04-23 14:02:52,113 INFO mapred.FileInputFormat: Total input files to process : 1
2024-04-23 14:02:52,150 INFO mapreduce.JobSubmitter: number of splits:1
2024-04-23 14:02:52,634 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local672430124_0001
2024-04-23 14:02:52,635 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-04-23 14:02:52,902 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
2024-04

If the `output` directory contains the empty file `_SUCCESS`, this means that the job was successful.

Check the output of the MapReduce job.

In [25]:
!hdfs dfs -cat output_simplest/part-00000

  10000   10000 18299443	


The number of words is in this case equal to the number of lines because there are no word separators (empty spaces) in the file, so each line is a word.

## Another MapReduce example: filter a log file

We're going to use a Linux logfile and look for the string `sshd` in a given position. The file stems from [Loghub](https://github.com/logpai/loghub), a freely available collection of system logs for AI-driven log analytics research.

The mapper `mapper.py` filters the file for the given string `sshd` at field 4.

The job has no reducer (option `-reducer NONE`). Note that without a reducer the sorting and shuffling phase after the map phase is skipped.


Download the logfile `Linux_2k.log`:

In [26]:
!wget --no-clobber https://raw.githubusercontent.com/logpai/loghub/master/Linux/Linux_2k.log

--2024-04-23 14:03:01--  https://raw.githubusercontent.com/logpai/loghub/master/Linux/Linux_2k.log
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 216485 (211K) [text/plain]
Saving to: ‘Linux_2k.log’

Linux_2k.log        100%[===================>] 211.41K  --.-KB/s    in 0.03s   

2024-04-23 14:03:01 (8.13 MB/s) - ‘Linux_2k.log’ saved [216485/216485]



In [27]:
%%bash
hdfs dfs -mkdir input || true
hdfs dfs -put Linux_2k.log input/ || true

Define the mapper

In [28]:
%%writefile mapper.py
#!/usr/bin/env python
import sys

for line in sys.stdin:
    # split the line into words
    line = line.strip()
    fields = line.split()
    if (len(fields)>=5 and fields[4].startswith('sshd')):
      print(line)


Writing mapper.py


Test the script (after setting the correct permissions)

In [29]:
!chmod 700 mapper.py

Look at the first 10 lines

In [30]:
!head -10 Linux_2k.log

Jun 14 15:16:01 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4 
Jun 14 15:16:02 combo sshd(pam_unix)[19937]: check pass; user unknown
Jun 14 15:16:02 combo sshd(pam_unix)[19937]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4 
Jun 15 02:04:59 combo sshd(pam_unix)[20882]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20884]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20883]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20885]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd

Test the mapper in the shell (not using MapReduce):

In [31]:
!head -100 Linux_2k.log| ./mapper.py

Jun 14 15:16:01 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4
Jun 14 15:16:02 combo sshd(pam_unix)[19937]: check pass; user unknown
Jun 14 15:16:02 combo sshd(pam_unix)[19937]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4
Jun 15 02:04:59 combo sshd(pam_unix)[20882]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20884]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20883]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20885]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(p

Now run the MapReduce job on the pseudo-cluster

In [32]:
%%bash

hdfs dfs -rm -r output_filter

mapred streaming \
  -file mapper.py \
  -input input \
  -output output_filter \
  -mapper mapper.py \
  -reducer NONE


packageJobJar: [mapper.py] [] /tmp/streamjob209205563533773013.jar tmpDir=null


rm: `output_filter': No such file or directory
2024-04-23 14:03:16,223 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
2024-04-23 14:03:17,995 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2024-04-23 14:03:18,193 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2024-04-23 14:03:18,193 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2024-04-23 14:03:18,220 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2024-04-23 14:03:18,827 INFO mapred.FileInputFormat: Total input files to process : 1
2024-04-23 14:03:18,852 INFO mapreduce.JobSubmitter: number of splits:1
2024-04-23 14:03:19,167 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1155608604_0001
2024-04-23 14:03:19,168 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-04-23 14:03:19,601 INFO mapred.LocalDistributedCacheManager: Localized file:/content/mapper.py as

Check the result

In [33]:
!hdfs dfs -ls output_filter

Found 2 items
-rw-r--r--   1 root supergroup          0 2024-04-23 14:03 output_filter/_SUCCESS
-rw-r--r--   1 root supergroup      85436 2024-04-23 14:03 output_filter/part-00000


In [34]:
!hdfs dfs -cat output_filter/part-00000 |head

Jun 14 15:16:01 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4	
Jun 14 15:16:02 combo sshd(pam_unix)[19937]: check pass; user unknown	
Jun 14 15:16:02 combo sshd(pam_unix)[19937]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4	
Jun 15 02:04:59 combo sshd(pam_unix)[20882]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root	
Jun 15 02:04:59 combo sshd(pam_unix)[20884]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root	
Jun 15 02:04:59 combo sshd(pam_unix)[20883]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root	
Jun 15 02:04:59 combo sshd(pam_unix)[20885]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root	
Jun 15 02:04:59 combo

## Aggregate data with MapReduce

Following the example in [Hadoop Streaming/Aggregate package](https://hadoop.apache.org/docs/stable/hadoop-streaming/HadoopStreaming.html#Hadoop_Aggregate_Package)

In [35]:
%%writefile myAggregatorForKeyCount.py
#!/usr/local/bin/python
import sys

def generateLongCountToken(id):
    return "LongValueSum:" + id + "\t" + "1"

def main(argv):
    line = sys.stdin.readline()
    try:
        while line:
            line = line[:-1]
            fields = line.split()
            s = fields[4].split('[')[0]
            print(generateLongCountToken(s))
            line = sys.stdin.readline()
    except "end of file":
        return None

if __name__ == "__main__":
     main(sys.argv)

Writing myAggregatorForKeyCount.py


Set permissions

In [36]:
!chmod 700 myAggregatorForKeyCount.py

Test the mapper

In [37]:
!head -20 Linux_2k.log| ./myAggregatorForKeyCount.py

LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:su(pam_unix)	1
LongValueSum:su(pam_unix)	1
LongValueSum:logrotate:	1
LongValueSum:su(pam_unix)	1
LongValueSum:su(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1


Run the MapReduce job

In [38]:
%%bash

hdfs dfs -rm -r output_aggregate

mapred streaming \
  -file mapper.py \
  -file myAggregatorForKeyCount.py \
  -input input \
  -output output_aggregate \
  -mapper myAggregatorForKeyCount.py \
  -reducer aggregate


packageJobJar: [mapper.py, myAggregatorForKeyCount.py] [] /tmp/streamjob10984543875680023726.jar tmpDir=null


rm: `output_aggregate': No such file or directory
2024-04-23 14:03:36,158 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
2024-04-23 14:03:39,272 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2024-04-23 14:03:39,691 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2024-04-23 14:03:39,691 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2024-04-23 14:03:39,753 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2024-04-23 14:03:40,344 INFO mapred.FileInputFormat: Total input files to process : 1
2024-04-23 14:03:40,377 INFO mapreduce.JobSubmitter: number of splits:1
2024-04-23 14:03:40,718 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1578171682_0001
2024-04-23 14:03:40,719 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-04-23 14:03:41,111 INFO mapred.LocalDistributedCacheManager: Localized file:/content/mapper.py

Check result

In [39]:
%%bash
hdfs dfs -ls output_aggregate
hdfs dfs -cat output_aggregate/part-00000

Found 2 items
-rw-r--r--   1 root supergroup          0 2024-04-23 14:03 output_aggregate/_SUCCESS
-rw-r--r--   1 root supergroup        326 2024-04-23 14:03 output_aggregate/part-00000
--	1
bluetooth:	2
cups:	12
ftpd	916
gdm(pam_unix)	2
gdm-binary	1
gpm	2
hcid	1
irqbalance:	1
kernel:	76
klogind	46
login(pam_unix)	2
logrotate:	43
named	16
network:	2
nfslock:	1
portmap:	1
random:	1
rc:	1
rpc.statd	1
rpcidmapd:	1
sdpd	1
snmpd	1
sshd(pam_unix)	677
su(pam_unix)	172
sysctl:	1
syslog:	2
syslogd	7
udev	8
xinetd	2


Pretty-print table of aggregated data

In [40]:
%%bash
hdfs dfs -get output_aggregate/part-00000 result # download results file
column -t result|sort -k2nr # sort by field 2 numerically in descending order

ftpd             916
sshd(pam_unix)   677
su(pam_unix)     172
kernel:          76
klogind          46
logrotate:       43
named            16
cups:            12
udev             8
syslogd          7
bluetooth:       2
gdm(pam_unix)    2
gpm              2
login(pam_unix)  2
network:         2
syslog:          2
xinetd           2
--               1
gdm-binary       1
hcid             1
irqbalance:      1
nfslock:         1
portmap:         1
random:          1
rc:              1
rpcidmapd:       1
rpc.statd        1
sdpd             1
snmpd            1
sysctl:          1


# Stop cluster

When you're done with your computations, you can shut down the Hadoop cluster and stop the `sshd` service.

In [41]:
!./hadoop-3.3.6/sbin/stop-dfs.sh

/bin/bash: line 1: ./hadoop-3.3.6/sbin/stop-dfs.sh: No such file or directory


Stop the `sshd` daemon

In [42]:
!/etc/init.d/ssh stop

 * Stopping OpenBSD Secure Shell server sshd
   ...done.


# Concluding remarks

We have started a single-node Hadoop cluster and ran some simple HDFS and MapReduce commands.

Even when running on a single machine, one can benefit from the parallelism provided by multiple virtual cores.

Hadoop provides also a command-line utility (the CLI MiniCluster) to start and stop a single-node Hadoop cluster "_without the need to set any environment variables or manage configuration files_" (https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/CLIMiniCluster.html). Alas, I was not able to get the MiniCluster to run due to some uncompatible libraries if I remember correctly.

If on one hand it would be great to be able to start a Hadoop cluster with a single command, it is interesting to see how the single components of a Hadoop cluster come into play and it helps to learn about the Hadoop architecture.

If you liked this notebook, check also:
 - [Hadoop single-node cluster setup with Python](https://github.com/groda/big_data/blob/master/Hadoop_single_node_cluster_setup_Python.ipynb) similar to this but using Python in place of bash
 - [Setting up Spark Standalone on Google Colab](https://github.com/groda/big_data/blob/master/Hadoop_Setting_up_Spark_Standalone_on_Google_Colab.ipynb)
 - [Getting to know the Spark Standalone Architecture](https://github.com/groda/big_data/blob/master/Spark_Standalone_Architecture_on_Google_Colab.ipynb)


